In [10]:
# Check if the connection already exists
if connections.has_connection('default'):
    connections.remove_connection('default')  # Disconnect if it exists

# Now, reconnect with your new configuration
connections.connect(alias='default', host='localhost', port='19530')

In [14]:
print(collection.schema)


{'auto_id': False, 'description': 'collection description of the collection: text_collection', 'fields': [{'name': 'uuid', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 36}, 'is_primary': True, 'auto_id': False}, {'name': 'partition_name', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'embeds', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'text_id', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'media', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}, {'name': 'link', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 5000}}]}


In [15]:
from pymilvus import Collection, connections

# Check if the connection already exists
if connections.has_connection('default'):
    connections.remove_connection('default')  # Disconnect if it exists

# Now, reconnect with your new configuration
connections.connect(alias='default', host='localhost', port='19530')

collection_name = "text_collection"

# Load the existing collection by its name
collection = Collection(name=collection_name)

# Load the collection into memory
collection.load()

query_text = 'Emiliano'
query_embedding = embed_string(query_text)

# Define the search parameters
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

# Perform the search
results = collection.search([query_embedding], "embeds", param=search_params, limit=10)

# Process the results
for hit in results[0]:
    print(f"ID: {hit.id}, Distance: {hit.distance}")


ID: 8c836938-f55f-4ffa-85c5-83bd64d83ed5, Distance: 0.534145712852478
ID: 5fd1ed2c-0d20-4345-8aa2-a124bce10b8d, Distance: 0.5446814894676208
ID: 5a8ce30b-8c7f-48ea-814f-54477bb0cbf5, Distance: 0.5455089807510376
ID: fa515dc6-a86b-4ed3-b81c-0f79ae5a030f, Distance: 0.5479221940040588
ID: 84866277-258e-4608-9e8e-68daa3f35997, Distance: 0.5485208034515381
ID: 1c6dde47-9824-4f63-9419-2572460d4a50, Distance: 0.5494377613067627
ID: cf710cef-40c0-4b09-9a5b-fd983b5b6f51, Distance: 0.5498582124710083
ID: 4fbb619e-3b4f-41a6-9782-da73d020fad8, Distance: 0.5519993305206299
ID: 28e065fb-bf33-450f-b0c7-c6c5db8a5de6, Distance: 0.5530645251274109
ID: 82e98d1b-a55d-4a91-8080-7fcf8d98e127, Distance: 0.5535456538200378


In [7]:
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

def embed_string(text: str):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-v2')
    model = AutoModel.from_pretrained('intfloat/e5-large-v2')

    # Prefix the text with 'query: '
    text = 'query: ' + text

    # Tokenize the input text
    inputs = tokenizer(text, max_length=2000, padding=True, truncation=True, return_tensors='pt')

    # Generate model outputs
    outputs = model(**inputs)

    # Average pool the last hidden states and apply the attention mask
    embeddings = average_pool(outputs.last_hidden_state, inputs['attention_mask'])

    # Normalize the embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)

    # Convert tensor to list
    embeddings_list = embeddings.tolist()

    return embeddings_list[0]